In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchaseData = pd.read_csv(file_to_load)

In [2]:
#Understand data
purchaseData.shape

(780, 7)

In [3]:
purchaseData.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
purchaseDf= purchaseData.copy()

In [5]:
#Total Number of Players
uniquePlayers=purchaseDf.SN.nunique() #find number of unique players
totalPlayers=pd.DataFrame()      #display players in DF
totalPlayers["Total Players"]=[uniquePlayers]
totalPlayers

,Total Players
0,576


# Purchasing Analysis (Total)

In [6]:
#Number of Unique Items
uniqueItems=purchaseDf["Item Name"].nunique()
uniqueItems

#Average Purchase Price
avePrice=purchaseDf.Price.mean()
avePrice

#Total Number of Purchases
totalPur=purchaseDf["Purchase ID"].nunique()
totalPur


#Total Revenue
totalRev=purchaseDf.Price.sum()
totalRev

#summary table
purchasingDf=pd.DataFrame()
purchasingDf["Number of Unique Items"]= [uniqueItems]
purchasingDf["Average Purchase Price"]= [avePrice]
purchasingDf["Total Number of Purchases"]= [totalPur]
purchasingDf["Total Revenue"]= [totalRev]

#formatting in summary display
#https://stackoverflow.com/questions/22086619/how-to-apply-a-function-to-multiple-columns-in-a-pandas-dataframe-at-one-time
cols1=["Average Purchase Price","Total Revenue"]
purchasingDf[cols1]=purchasingDf[cols1].applymap("${:,.2f}".format)

#display summary table
purchasingDf

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


# Gender Demographics

In [7]:
#display options under Gender column
genders=purchaseDf.Gender.unique()
genders

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [8]:
players= purchaseDf.groupby(["Gender","SN"]).size() #gather unique players in groups by gender

#Count of Players by gender
male= players["Male"].count()
female=players["Female"].count()
other= players["Other / Non-Disclosed"].count()

#Percentage of Players by gender
malePerc= (male/uniquePlayers)*100
femalePerc= (female/uniquePlayers)*100
otherPerc = (other/uniquePlayers)*100

#summary table
genderDf=pd.DataFrame()
genderDf["Genders"]=["Male","Female","Other / Non-Disclosed"]
genderDf["Count"]=[male,female,other]
genderDf["Percent"]=[malePerc,femalePerc,otherPerc]

#summary table formatting
genderDf["Percent"]=genderDf["Percent"].map("{:.2f}%".format)

genderDf

,Genders,Count,Percent
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


# Purchasing Analysis (Gender)

In [9]:
#purchase count, average purchase price, and total purchase value in summary display

#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html  (referenced for more detail on .agg function)
#https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas (referenced to rename columns on table)

perAveTot= purchaseDf.groupby(["Gender"]).agg({"Purchase ID": "count", "Price":["mean","sum"]})
perAveTot.columns=["Number of Purchases","Average Purchase Price","Total Purchase Value"]

#Average Purchase Total per Person by Gender
onePerson= purchaseDf.groupby(["Gender","SN"]).agg({"Price":"sum"})
purchaseGen= onePerson.groupby(["Gender"]).mean()
purchaseGen.columns=["Average Purchase Total per Person by Gender"]

#append/merge Average Purchase Total per Person by Gender to summary display
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
byGender=pd.merge(perAveTot,purchaseGen,how="inner",on="Gender")

#formatting in summary display
cols2=["Average Purchase Price","Total Purchase Value","Average Purchase Total per Person by Gender"]
byGender[cols2]=byGender[cols2].applymap("${:,.2f}".format)

#display summary table
byGender


,Number of Purchases,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


# Age Demographics

In [10]:
# Create bins for age categories
bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
# Create labels for these bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [11]:
#create new dataframe and add row of bins
ageDf= purchaseDf.copy()
ageDf["Age_Group"]= pd.cut(purchaseDf["Age"], bins, labels=group_labels)
ageDf.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age_Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [12]:
#count and percentage by age bin
ageNum= ageDf.groupby(["Age_Group"]).agg({"SN":"nunique"}) #gather unique players in groups by age bin
agePerc= (ageNum / uniquePlayers)*100

#append/merge count and perc to summary display and fix column Names
byAge=pd.merge(ageNum,agePerc,how="inner",on="Age_Group")
byAge.columns=["Count","Percent"]

#summary table formatting
byAge["Percent"]=byAge["Percent"].map("{:.2f}%".format)

#display summary table
byAge

,Count,Percent
Age_Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


# Purchasing Analysis (Age)

In [13]:
#calculations on new data frame for metrics by age bin
perAveTot2= ageDf.groupby(["Age_Group"]).agg({"Purchase ID": "count", "Price":["mean","sum"]})
perAveTot2.columns=["Number of Purchases","Average Purchase Price","Total Purchase Value"]


#Average Purchase Total per Person by Age
byPerson=ageDf.groupby(["Age_Group"]).agg({"SN":"nunique","Price":"sum"}) #number of unique players and sum of purchases in each age group
perPerson=byPerson.Price/byPerson.SN #average
avePerPerson=pd.DataFrame(perPerson) #display results in DataFrame
avePerPerson.columns=["Average Purchase Total per Person"]
avePerPerson

#append/merge count and perc to summary display
purchAge=pd.merge(perAveTot2,avePerPerson,how="inner",on="Age_Group")

#formatting in summary display
cols4=["Average Purchase Price","Total Purchase Value","Average Purchase Total per Person"]
purchAge[cols4]=purchAge[cols4].applymap("${:,.2f}".format)

#display summary table
purchAge



,Number of Purchases,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age_Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


# Top Spenders

In [14]:
#identify top spenders
#https://datatofish.com/sort-pandas-dataframe/
topSpend= purchaseDf.groupby(["SN"]).agg({"Purchase ID": "count","Price": ["mean","sum"]})
topSpend.columns=["Purchase Count","Average Purchase Price","Total Purchase Value"]
topSpend.sort_values(by=["Total Purchase Value"], inplace=True, ascending=False)

#formatting in summary table
cols3=["Average Purchase Price","Total Purchase Value"]
topSpend[cols3]=topSpend[cols3].applymap("${:.2f}".format)

#display top 5 of summary table
topSpend.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [24]:
#find how many players are one time shoppers
mask=topSpend["Purchase Count"]==1
oneTime= topSpend.loc[mask]

#find out percentage of players are one-time shoppers
len(oneTime)/len(topSpend)

0.71875

# Most Popular Items

In [15]:
#identify most profitable items
mostPop= purchaseDf.groupby(["Item ID","Item Name"]).agg({"Purchase ID": "count","Price":["mean", "sum"]})
mostPop.columns=["Purchase Count", "Item Price","Total Purchase Value"]
mostPop.sort_values(by=["Purchase Count"], inplace=True, ascending=False)

#formatting in summary table
cols5=["Item Price","Total Purchase Value"]
mostPop[cols5]=mostPop[cols5].applymap("${:.2f}".format)

#display top 5 items
mostPop.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


# Most Profitable Items

In [17]:
#identify most profitable items
mostProf= purchaseDf.groupby(["Item ID","Item Name"]).agg({"Purchase ID": "count","Price":["mean", "sum"]})
mostProf.columns=["Purchase Count", "Item Price","Total Purchase Value"]
mostProf.sort_values(by=["Total Purchase Value"], inplace=True, ascending=False)

#formatting in summary table
cols6=["Item Price","Total Purchase Value"]
mostProf[cols6]=mostProf[cols6].applymap("${:.2f}".format)

#display top 5 items
mostProf.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


Resources:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
https://datatofish.com/sort-pandas-dataframe/
https://stackoverflow.com/questions/22086619/how-to-apply-a-function-to-multiple-columns-in-a-pandas-dataframe-at-one-time